In [1]:
# import dependencies
import spektral
import tensorflow as tf
from spektral.layers import GraphConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout

In [2]:
# load cora data set
cora_A, cora_X, cora_L, cora_train_mask, cora_val_mask, cora_test_mask = spektral.datasets.citation.load_data(dataset_name='cora',
                                                                                                              normalize_features=True, 
                                                                                                              random_split=False)

Loading cora dataset
Pre-processing node features


cora_A - adjacency matrix

cora_X - feature matrix (doc by term)

cora_L - label

In [3]:
# define number of nodes, features and classes
num_cora_nodes = cora_A.shape[0]
num_cora_features = cora_X.shape[1]
num_cora_classes = cora_L.shape[1]

In [4]:
# print out attributes
print('shape of cora Adjacency Matrix: {} x {}'.format(num_cora_nodes, num_cora_nodes))
print('number of cora features (number of termrs): ', num_cora_features)
print('number of cora classes: ', num_cora_classes)

shape of cora Adjacency Matrix: 2708 x 2708
number of cora features (number of termrs):  1433
number of cora classes:  7


In [5]:
# check the distribution of each class for balance
cora_L.sum(axis = 0)

array([351, 217, 418, 818, 426, 298, 180])

In [6]:
# define feature and adjacency input to the first GCN layer
X_in = Input(shape = (num_cora_features, ))
A_in = Input(shape = (num_cora_nodes, ), sparse = True)
# construct 3 layers of GCN, features beuing reduced to
# 64, 32 and 7 (number of classes)
# use drop out of 0.5 to minimize overfitting
X_1 = GraphConv(64, 'relu',)([X_in, A_in])
X_1 = Dropout(0.5)(X_1)
X_2 = GraphConv(32, 'relu')([X_1, A_in])
X_2 = Dropout(0.5)(X_2)
X_3 = GraphConv(num_cora_classes, 'softmax')([X_2, A_in])
# use keras functional API to construct a GCN model
cora_model = Model(inputs = [X_in, A_in], outputs = X_3, name = 'cora_GCN_model')

In [7]:
# preprocess adjacency matrix to add self-loops and scale edge weights
cora_A = GraphConv.preprocess(cora_A).astype('f4')

In [8]:
# compile model
cora_model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             weighted_metrics = ['acc', tf.keras.metrics.AUC()])
cora_model.summary()

Model: "cora_GCN_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv (GraphConv)          (None, 64)           91776       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 64)           0           graph_conv[0][0]    

In [9]:
# define number of epochs
cora_epochs = 100
# Prepare data
cora_X = cora_X.toarray()

In [10]:
# prepare val data
cora_val_data = ([cora_X, cora_A], cora_L, cora_val_mask)

# Train model
cora_model.fit([cora_X, cora_A], cora_L,
               sample_weight = cora_train_mask,
               validation_data = cora_val_data,
               epochs = cora_epochs,
               batch_size = num_cora_nodes,
               shuffle = False)

Epoch 1/100
1/1 [==============================] - 1s 951ms/step - loss: 0.1006 - acc: 0.1429 - auc: 0.5000 - val_loss: 0.3591 - val_acc: 0.2340 - val_auc: 0.5012
Epoch 2/100
1/1 [==============================] - 0s 459ms/step - loss: 0.1005 - acc: 0.1500 - auc: 0.5012 - val_loss: 0.3590 - val_acc: 0.2780 - val_auc: 0.5017
Epoch 3/100
1/1 [==============================] - 0s 429ms/step - loss: 0.1005 - acc: 0.2143 - auc: 0.5036 - val_loss: 0.3589 - val_acc: 0.3180 - val_auc: 0.5040
Epoch 4/100
1/1 [==============================] - 0s 488ms/step - loss: 0.1005 - acc: 0.2357 - auc: 0.5077 - val_loss: 0.3588 - val_acc: 0.3460 - val_auc: 0.5053
Epoch 5/100
1/1 [==============================] - 0s 493ms/step - loss: 0.1004 - acc: 0.2714 - auc: 0.5118 - val_loss: 0.3587 - val_acc: 0.3520 - val_auc: 0.5085
Epoch 6/100
1/1 [==============================] - 0s 441ms/step - loss: 0.1004 - acc: 0.3429 - auc: 0.5298 - val_loss: 0.3585 - val_acc: 0.3600 - val_auc: 0.5159
Epoch 7/100
1/1 [=====

In [11]:
# Evaluate model
eval_results = cora_model.evaluate([cora_X, cora_A], 
                                   cora_L,
                                   sample_weight = cora_test_mask,
                                   batch_size = num_cora_nodes)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

1/1 [==============================] - 0s 1ms/step - loss: 0.5619 - acc: 0.7460 - auc: 0.9231
Done.
Test loss: 0.5618841052055359
Test accuracy: 0.7459999918937683
